In [2]:
import ftplib
import os
import py7zr
import pandas as pd
import numpy as np

In [ ]:
# CONNECT TO FTP
ftp_host = 'ftp.mtps.gov.br'
ftp = ftplib.FTP(ftp_host)
ftp.login()
ftp.cwd('pdet/microdados/CAGED/')

In [ ]:
# DECLARE A FUNCTION TO DOWNLOAD FILES FROM FTP SERVER 
# AND EXTRACT(.7z) THEM TO A TEMPORARY FOLDER
def downloadAndExtract(ftp, filename, path):
    tempfile_path = path + "/tmp/" + filename
    extracted_path = path + "/tmp/extracted"
    # print(tempfile_path, "\n", extracted_path)
    with open(tempfile_path, 'wb') as f:
        ftp.retrbinary(f'RETR {filename}', f.write)
    py7zr.SevenZipFile(tempfile_path, mode='r').extractall(path=extracted_path)

In [ ]:
#FILTER FOLDERS BY YEAR AND CALL DOWNALOADANDEXTRACT FUNCTION
filter = ['2019', '2018']
path = os.getcwd()
for dir in filter:
    ftp.cwd(dir)
    files = []
    ftp.retrlines('LIST', files.append)
    for file in files:
        downloadAndExtract(ftp, file.split()[3], path)
        # print(file.split()[3])
    ftp.cwd('..')

In [ ]:
#READ .TXT FILES AS .CSV AND CONCATENATE THEM INTO A SINGLE PANDAS DATAFRAME
path = os.getcwd() + "/tmp/extracted/"
dataFrames=[]
for root, dirs, files in os.walk(path):
    for file in files:
        dfTmp= pd.read_csv(path + file, encoding='latin-1', sep=';')
        dataFrames.append(dfTmp)
df = pd.concat(dataFrames, ignore_index=True)

In [ ]:
# HERE WE START TO EXPLORE AND CLEAN THE DATA
df.info()

In [ ]:
# CLEANING AND FORMATING COLUMNS THAT HAD DATA TYPE 'Object'
df.loc[df['Grau Instrução']=='{ñ','Grau Instrução'] = 7
df['Salário Mensal'] = df['Salário Mensal'].str.replace(',','.')
df.loc[df['Bairros SP']=='{ñ c','Bairros SP'] = 0
df.loc[df['Bairros Fortaleza']=='{ñ c','Bairros Fortaleza'] = 0
df.loc[df['Bairros RJ']=='{ñ c','Bairros RJ'] = 0
df.loc[df['Distritos SP']=='{ñ c','Distritos SP'] = 0
df.loc[df['Região Adm RJ']=='{ñ','Região Adm RJ'] = 0
df.loc[df['Região Adm SP']=='{ñ ','Região Adm SP'] = 0
df.loc[df['Região Corede']=='{ñ','Região Corede'] = 0
df.loc[df['Região Corede 04']=='{ñ','Região Corede 04'] = 0
df.loc[df['Região Gov SP'].isin(['{ñ ','***']),'Região Gov SP'] = 0
df.loc[df['Região Senac PR'].isin(['{ñ ','{ñ','***']),'Região Senac PR'] = 0
df.loc[df['Região Senai PR'].isin(['{ñ ','{ñ','***']),'Região Senai PR'] = 0
df.loc[df['Região Senai SP'].isin(['{ñ ','{ñ','***']),'Região Senai SP'] = 0
df.loc[df['Sub-Região Senai PR'].isin(['{ñ ','{ñ','***']),'Sub-Região Senai PR'] = 0
df['Tempo Emprego'] = df['Tempo Emprego'].astype('float64')
df['Tempo Emprego'] = df['Tempo Emprego'].astype('int64')

In [ ]:
# CHANGING DATA TYPES
df['Grau Instrução'] = df['Grau Instrução'].astype('int64')
df['Salário Mensal'] = df['Salário Mensal'].astype('float64')
df['Bairros SP'] = df['Bairros SP'].astype('int64')
df['Bairros Fortaleza'] = df['Bairros Fortaleza'].astype('int64')
df['Bairros RJ'] = df['Bairros RJ'].astype('int64')
df['Distritos SP'] = df['Distritos SP'].astype('int64')
df['Região Adm RJ'] = df['Região Adm RJ'].astype('int64')
df['Região Adm SP'] = df['Região Adm SP'].astype('int64')
df['Região Corede'] = df['Região Corede'].astype('int64')
df['Região Corede 04'] = df['Região Corede 04'].astype('int64')
df['Região Gov SP'] = df['Região Gov SP'].astype('int64')
df['Região Senac PR'] = df['Região Senac PR'].astype('int64')
df['Região Senai PR'] = df['Região Senai PR'].astype('int64')
df['Região Senai SP'] = df['Região Senai SP'].astype('int64')
df['Sub-Região Senai PR'] = df['Sub-Região Senai PR'].astype('int64')

In [ ]:
# df['Região Adm SP'].value_counts()
# df['Grau Instrução'].isna().sum()
# df['Salário Mensal'].groupby(df['Salário Mensal']).count()
# df['Distritos SP'].value_counts()
# df.iloc[:,31:39].info()

In [ ]:
# SAVE DATAFRAME TO PARQUET FILE
df.to_parquet('caged.parquet')

In [ ]:
# RECHARGE DATAFRAME FROM PARQUET FILE
df = pd.read_parquet('caged.parquet')
df.info()

In [9]:
# CREATE A DATAFRAME FROM THE Municipio TABLE
df_mun = pd.read_excel('./tmp/CAGEDEST_layout_Atualizado.xls', sheet_name= 'municipio')


In [12]:
# SPLIT THE COLUM MUNICIPIO INTO THREE COLUMNS:'Município', 'MunicípioStr' AND 'UFStr' BY THE ":" AND "-" SEPARATORS
df_mun[['Município', 'MunicípioStr']] = df_mun['Município'].str.split(':', expand=True)
df_mun[['UFStr', 'MunicípioStr']] = df_mun['MunicípioStr'].str.split('-', expand=True, n=1)
df_mun.head()

,Município,MunicípioStr,UFStr
0,110001,Alta Floresta D Oeste,Ro
1,110002,Ariquemes,Ro
2,110003,Cabixi,Ro
3,110004,Cacoal,Ro
4,110005,Cerejeiras,Ro


In [19]:
# CHANGING DATA TYPES
df_mun['Município'] = df_mun['Município'].astype('int64')
df_mun['UFStr'] = df_mun['UFStr'].astype('str')
df_mun['MunicípioStr'] = df_mun['MunicípioStr'].astype('str')


In [20]:
# MERGE THE DATAFRAMES 'df' AND 'df_mun' BY THE COLUMN 'Município'
df = pd.merge(df, df_mun, left_on='Município', right_on='Município', how='left')
df.head()

In [23]:
# FILTER THE DATAFRAME TO INCLUDE ONLY THE CAPITALS
capitals = ['Rio Branco', 'Maceió', 'Macapá', 'Manaus', 'Salvador', 'Fortaleza',
            'Brasília', 'Vitória', 'Goiânia', 'São Luís', 'Cuiabá', 'Campo Grande',
            'Belo Horizonte', 'Belém', 'João Pessoa', 'Curitiba', 'Recife', 'Teresina',
            'Rio de Janeiro', 'Natal', 'Porto Alegre', 'Porto Velho', 'Boa Vista',
            'Florianópolis', 'São Paulo', 'Aracaju', 'Palmas']
df = df.loc[df['MunicípioStr'].isin(capitals),:]
df.head(20)

In [27]:
# SAVE DATAFRAME TO PARQUET AND CSV FILE

df.to_parquet('caged_capitals.parquet')
df.to_csv('caged_capitals.csv')